# Aprendizado Supervisionado e K Vizinhos mais Próximos

## Introdução

Nós utilizaremos a base de dados de taxa de rejeição da indústria de telecomunicações para essa lista de exercícios. A base de dados tem o nome `Orange_Telecom_Churn_Data.csv`. Nesse notebook carregaremos a base de dados, faremos algum pré-processamento e usaremos a técnica k-NN para predizer a taxa de rejeição baseado nas características mensuradas.

## Exercício 1

* Comece importando os dados através do pandas. Examine as colunas e os dados
* Note que os dados contêm estado, código de área e telefone. Você acha que esses atributos são interessantes para construir nosso modelo de classificação? Por que?

Não utilizaremos esses atributos para essa base, então eles podem ser removidos do dataframe (método `drop`).

In [15]:
import pandas as pd

arquivo = 'C:/Users/StiphanieMaradaSilva/Desktop/PROJETOS/AprendizadoMaquina-master/UFABC-ML-2018/data/Orange_Telecom_Churn_Data.csv'
data = pd.read_csv(arquivo)
print(data.info())
data = data.drop(['state','area_code','phone_number'],axis=1)
print(data['churned'].value_counts())

#X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#X_scaled = X_std * (max - min) + min

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 21 columns):
state                            5000 non-null object
account_length                   5000 non-null int64
area_code                        5000 non-null int64
phone_number                     5000 non-null object
intl_plan                        5000 non-null object
voice_mail_plan                  5000 non-null object
number_vmail_messages            5000 non-null int64
total_day_minutes                5000 non-null float64
total_day_calls                  5000 non-null int64
total_day_charge                 5000 non-null float64
total_eve_minutes                5000 non-null float64
total_eve_calls                  5000 non-null int64
total_eve_charge                 5000 non-null float64
total_night_minutes              5000 non-null float64
total_night_calls                5000 non-null int64
total_night_charge               5000 non-null float64
total_intl_minutes           

## Exercício 2

* Note que algumas das colunas são categóricas e algumas são *float*. Esses atributos precisam ser numéricos para usar os algoritmos que aprenderemos no curso.
* Finalmente, o algoritmo K-Vizinhos mais próximos necessita de dados escalonados. Escalone os dados utilizando um dos métodos aprendidos em aula.

In [26]:
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
data_array = {}

bin_cols = ['intl_plan', 'voice_mail_plan', 'churned']
for col in data.columns:
    if col in bin_cols:
        obj = LabelBinarizer()
    else:
        obj = MinMaxScaler()
    d = obj.fit_transform(data[[col]])
    data_array[col] = d

print(data_array)

{'account_length': array([[ 0.52479339],
       [ 0.43801653],
       [ 0.56198347],
       ..., 
       [ 0.24793388],
       [ 0.44628099],
       [ 0.35123967]]), 'intl_plan': array([[0],
       [0],
       [0],
       ..., 
       [0],
       [0],
       [0]]), 'voice_mail_plan': array([[1],
       [1],
       [0],
       ..., 
       [0],
       [0],
       [1]]), 'number_vmail_messages': array([[ 0.48076923],
       [ 0.5       ],
       [ 0.        ],
       ..., 
       [ 0.        ],
       [ 0.        ],
       [ 0.65384615]]), 'total_day_minutes': array([[ 0.7541963 ],
       [ 0.45974395],
       [ 0.69246088],
       ..., 
       [ 0.4       ],
       [ 0.5371266 ],
       [ 0.36813656]]), 'total_day_calls': array([[ 0.66666667],
       [ 0.74545455],
       [ 0.69090909],
       ..., 
       [ 0.53939394],
       [ 0.40606061],
       [ 0.61818182]]), 'total_day_charge': array([[ 0.7541834 ],
       [ 0.45967202],
       [ 0.69243641],
       ..., 
       [ 0.39993307],
 

## Exercício 3

* Separe as colunas de atributos (todas menos `churned`) da  coluna de rótulo (`churned`). Isso criará duas tabelas.
* Aplique o método `.fit()` do K-nearest neighbors com um valor de `n_neighbors=3` para essa base de dados e verifique o resultado com o método `.predict()` na mesma base.

In [46]:
import numpy as np
# ignore churned column
x_cols = [x for x in data.columns if x != 'churned']

X_data = np.array([data_array[x].flatten() for x in x_cols])
y_data = data_array['churned'].flatten()

print(X_data.shape)

X_data = np.swapaxes(X_data, 0, 1)

[[ 0.52479339  0.43801653  0.56198347 ...,  0.24793388  0.44628099
   0.35123967]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 1.          1.          0.         ...,  0.          0.          1.        ]
 ..., 
 [ 0.15        0.15        0.25       ...,  0.2         0.3         0.8       ]
 [ 0.5         0.68518519  0.60925926 ...,  0.67962963  0.42592593
   0.46481481]
 [ 0.11111111  0.11111111  0.         ...,  0.11111111  0.          0.        ]]
(17, 5000)


In [58]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
features = np.reshape(y_data, (-1, 1))

model.fit(X_data, features)
predict = model.predict(X_data)


C:\Users\StiphanieMaradaSilva\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


## Exercício 4

Ainda não ensinamos formas de medir o erro, mas a acurácia é um conceito simples de entender--é a porcentagem de rótulos que foram corretamente classificados:

$acc = \frac{corretos}{total}$

* Escreva uma função para calcular a acurácia usando os rótulos verdadeiros e os preditos.
* Usando a função, calcule a acurácia do K-nn nos dados.

In [60]:
from sklearn.metrics import confusion_matrix    

def accuracy(real, predict):
    confusion = confusion_matrix(real, predict)
    correct = confusion[0,0] + confusion[1,1]
    wrong = confusion[0,1] + confusion[1,0]
    return correct / (correct + wrong)
    pass # substitua pelo código
print(accuracy(data['churned'], predict))

0.9422


## Exercício 5

* Repita a aplicação do modelo K-nearest neighbors mas com o parâmetro `weights=distance`. Calcule a acurácia da função criada por você.
* Repita a aplicação do modelo com `n_neighbors=3`, `weights=uniform` e `p=1` para utilizar a distância de Manhattan. Verifique os resultados.

Quando as distâncias ponderadas são utilizadas para a parte 1 dessa questão, vocês obterão acurácia de 1.0. Por que isso ocorre? *Dica:* o KNN usa diretamente os dados para fazer a predição.

## Exercício 6

* Teste o modelo KNN utilizando valores de `n_neighbors` na faixa de 1 a 20. Deixe o restante dos parâmetros como o padrão. Armazene os valores de vizinhos e acurácia em uma lista no formato [(k, acuracia)].
* Plote um gráfico do tipo *scatter* da acurácia vs vizinhos. O que acontece com `n_neighbors=1`? Por que isso ocorre?

In [15]:
score_list = list()

'''escreva no bloco abaixo'''








'''fim do código'''

score_df = pd.DataFrame(score_list, columns=['k', 'accuracy'])

In [16]:
# Import libraries to make the plot

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [17]:
sns.set_context('talk')
sns.set_style('ticks')
sns.set_palette('dark')

ax = score_df.set_index('k').plot()

ax.set(xlabel='k', ylabel='accuracy')
ax.set_xticks(range(1, 21));

TypeError: Empty 'DataFrame': no numeric data to plot